In [1]:
import enum
import pathlib
from typing import TypeAlias
from dataclasses import dataclass

from sqlalchemy import (
    Column,
    Enum,
    Integer,
    String,
    Table
)
from sqlalchemy.orm import registry

import vocalpy as voc

In [2]:
cd /Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy

/Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy


In [3]:
@dataclass
class AudioFile:
    path: pathlib.Path

In [4]:
@dataclass
class SpectrogramParameters:
    fft_size: int
    step_size: int

In [5]:
@dataclass
class SpectrogramFile:
    path: pathlib.Path
    spectrogram_parameters: SpectrogramParameters
    source_audio_file: AudioFile

In [6]:
AnnotatedFile: TypeAlias = AudioFile | SpectrogramFile

@dataclass
class AnnotationFile:
    path: pathlib.Path
    annotates: AnnotatedFile

In [7]:
SourceFile: TypeAlias = AudioFile | SpectrogramFile | AnnotationFile

@dataclass
class FeatureFile:
    path: pathlib.Path
    source_file: SourceFile

In [8]:
class DatasetFileTypeEnum(enum.Enum):
    AudioFile = AudioFile
    SpectrogramFile = SpectrogramFile
    AnnotationFile = AnnotatedFile
    FeatureFile = FeatureFile


@dataclass
class DatasetFile:
    path: pathlib.Path
    filetype: DatasetFileTypeEnum

In [9]:
mapper_registry = registry()

In [10]:
dataset_file_table = Table(
    "dataset_file",
    mapper_registry.metadata,
    Column('id', Integer, primary_key=True),
    Column('filetype', Enum(DatasetFileTypeEnum)),
    Column('path', String),
)

In [11]:
mapper_registry.map_imperatively(DatasetFile, dataset_file_table)

<Mapper at 0x124ceccd0; DatasetFile>

In [12]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

mapper_registry.metadata.create_all(engine)

2023-04-18 21:08:17,208 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-18 21:08:17,209 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("dataset_file")
2023-04-18 21:08:17,209 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-18 21:08:17,211 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("dataset_file")
2023-04-18 21:08:17,212 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-18 21:08:17,213 INFO sqlalchemy.engine.Engine 
CREATE TABLE dataset_file (
	id INTEGER NOT NULL, 
	filetype VARCHAR(15), 
	path VARCHAR, 
	PRIMARY KEY (id)
)


2023-04-18 21:08:17,214 INFO sqlalchemy.engine.Engine [no key 0.00118s] ()
2023-04-18 21:08:17,217 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
cbin_paths = voc.paths.from_dir('./tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/', 'cbin')
audio_files = [AudioFile(path=cbin_path) for cbin_path in cbin_paths]

In [14]:
dataset_files = []
for audio_file in audio_files:
    dataset_file = DatasetFile(
        # file=audio_file,
        filetype=DatasetFileTypeEnum(type(audio_file)),
        path=audio_file.path
    )
    dataset_files.append(dataset_file)

In [15]:
dataset_files[0]

DatasetFile(path=PosixPath('tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0808.138.cbin'), filetype=<DatasetFileTypeEnum.AudioFile: <class '__main__.AudioFile'>>)

In [16]:
session.add(dataset_files[0])

In [17]:
session.query(DatasetFile).first()

2023-04-18 21:08:30,496 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-18 21:08:30,499 INFO sqlalchemy.engine.Engine INSERT INTO dataset_file (filetype, path) VALUES (?, ?)
2023-04-18 21:08:30,501 INFO sqlalchemy.engine.Engine [generated in 0.00168s] ('AudioFile', PosixPath('tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0808.138.cbin'))
2023-04-18 21:08:30,502 INFO sqlalchemy.engine.Engine ROLLBACK


InterfaceError: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.InterfaceError) Error binding parameter 1 - probably unsupported type.
[SQL: INSERT INTO dataset_file (filetype, path) VALUES (?, ?)]
[parameters: ('AudioFile', PosixPath('tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0808.138.cbin'))]
(Background on this error at: https://sqlalche.me/e/14/rvf5)